In [1]:
#!pip install accelerate==0.25.0
#!pip install bertopic==0.15.0
#!pip install datasets==2.14.4
#!pip install faiss-cpu==1.7.4
#!pip install langchain==0.0.348
#!pip install langchainhub==0.1.14
#!pip install sentence-transformers==2.2.2
#!pip install sentencepiece==0.1.99
#!pip install transformers==4.24.0
!pip install wandb --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 59.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: wandb
    Found existing installation: wandb 0.16.6
    Uninstalling wandb-0.16.6:
      Successfully uninstalled wandb-0.16.6


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import transformers
import typing as tp
import wandb

from transformers import AutoTokenizer, AutoModelForSequenceClassification, PreTrainedTokenizer, PreTrainedTokenizerFast
from datasets import load_dataset
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau, ExponentialLR
from torch.utils.data import DataLoader, Dataset

from tqdm import tqdm

from sklearn.metrics import recall_score, f1_score, precision_score
from pprint import pprint

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
torch.manual_seed(42)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [16]:
class CustomDataset(Dataset):
    """
    This class designs logic to retrieve data from a custom dataset.
    According to pytorch Dataset conception any map style dataset
    should implement at least __len__ and __getitem__ methods.
    """

    def __init__(
        self, texts, labels, tokenizer, max_length
    ) -> None:
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self) -> int:
        """
        returns number of rows in data
        """
        return len(self.texts)

    def __getitem__(self, idx: int) -> tp.Dict[str, tp.Any]:
        """
        retrieves data for single index.
        may include data processing and transformations.
        E.g. augmenting data or tokenizing it.
        returns dict with keys "input_ids", "label" and probably some more metadata (you decide whethere you need something more here)
        """
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'label': torch.tensor(label, dtype=torch.long)
        }


class ModelTrainer:
    """
    This class implements logic run an experiemnt with a provided transformers classification model.
    It incudes following components:
    - load data
    - load and configure a model and its artifacts
    - train model
    - validate model
    - save model
    - compue metrics
    - run_experiment (as the man entrypoint to execute all flow)

    Attention: current module intentionally doesnt support model inference or model serving.
    It is a good practice to separate train/inference classes otherwise it is hard to maintain it all.

    """

    def __init__(self, model_name: str, dataset_name: str, sample_size: int =-1, random_state: int = 42) -> None:
        self.model_name = model_name
        self.dataset_name = dataset_name
        
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        self.model = transformers.AutoModelForSequenceClassification.from_pretrained(self.model_name, num_labels=2)
        
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(self.model_name)
        self.model_config = transformers.AutoConfig.from_pretrained(self.model_name)
        self.sample_size = sample_size
        self.random_state = random_state

    def configure_optimizer(self, optimizer, params: tp.Dict) -> None:
        """
        adds a self.optimizer attribute with a chosen optimizer and its params.
        """

        self.optimizer = optimizer
        for key, value in params.items():
            for g in self.optimizer.param_groups:
                g[key] = value
                
        #pprint(optimizer.param_groups)

    def configure_scheduler(self, scheduler) -> None:
        """
        adds a self.scheduler attribute with a chosen scheduler (e.g. ReduceLROnPlateau).
        """

        self.scheduler = scheduler

    def apply_data_parallel(self) -> None:
        """
        checks number of available cuda devices,
        if number of GPUs is > 1, moves self.model to a DataParallel state for faster training.
        """
        
        if torch.cuda.device_count()>1:
            self.model = torch.nn.DataParallel(self.model, device_ids=list(range(torch.cuda.device_count())))

    def load_data(self, filename: str, split: str, sample_size: int = -1) -> pd.DataFrame:
        """
        uses Datasets library to load a dataset, takes as input dataset name (e.g. "imdb")
        and a split. Loads data into pandas.
        """
        
        ds = load_dataset(filename, split = split)
        if sample_size> 0 :
            return ds.to_pandas().sample(n = sample_size, random_state = self.random_state)
        else:
            return ds.to_pandas()
        
    def train(self, dataset: CustomDataset) -> None:
        
        # заглушка, если шедулер и оптимайзер не инцииализированы
        self.optimizer = self.optimizer or AdamW(self.model.parameters(), lr = 1e-3)
        self.scheduler = self.scheduler or ExponentialLR(optimizer=self.optimizer, gamma=0.9, last_epoch=-1)
        
        train_loader = DataLoader(dataset, batch_size=8, shuffle=False)
        
        self.model.to(self.device)
        
        correct = 0
        self.model.train()
        for epoch in tqdm(range(wandb.config["epochs"])):
            total_loss = 0.0

            for batch in train_loader:
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                labels = batch['label'].to(self.device)

                self.optimizer.zero_grad()
                outputs = self.model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                loss.backward()
                self.optimizer.step()
                
                
                #correct += (outputs[0].detach().cpu().numpy()==labels.to('cpu').numpy()).sum().item()
                #pprint(correct)
                total_loss += loss.item()
            if epoch % wandb.config["learning_rate_decay_step"] == 0:    
                self.scheduler.step()
            average_loss = total_loss / len(train_loader)
            #average_accuracy = correct / len(train_loader)
            #wandb.log({"accuracy": average_accuracy, "loss": average_loss})
            wandb.log({"loss": average_loss, "learning_rate": self.scheduler.get_last_lr()[0]})
            #print(f'Epoch {epoch+1}/{wandb.config["epochs"]} - Average Loss: {average_loss:.4f} - Average accuracy {average_accuracy:.4f}')
            print(f'Epoch {epoch+1}/{wandb.config["epochs"]} - Average Loss: {average_loss:.4f} - Learning Rate {self.scheduler.get_last_lr()[0]}')

        print("Training complete!")

    def validate(self, dataset: CustomDataset) -> tp.Dict[str, tp.Iterable]:
        """
        takes a trained model and runs it on validation data.
        Returns a dict with the keys "valid_labels" and "valid_preds" and corresponding values.
        """
        
        dataloader = DataLoader(dataset, batch_size=16, shuffle=False)
        
        self.model.to(self.device)
        self.model.eval()
        
        valid_preds, valid_labels = [], []

        for batch in dataloader:

            b_input_ids = batch["input_ids"].to(self.device)
            b_input_mask = batch["attention_mask"].to(self.device)
            b_labels = batch["label"].to(self.device)

            with torch.no_grad():
                logits = self.model(input_ids=b_input_ids, attention_mask=b_input_mask)

            logits = logits[0].detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            batch_preds = np.argmax(logits, axis=1)
            batch_labels = np.concatenate(label_ids.reshape(-1,1))
            valid_preds.extend(batch_preds)
            valid_labels.extend(batch_labels)

        return valid_labels, valid_preds

    def compute_metrics_report(
        self, labels: tp.Iterable, predictions: tp.Iterable
    ) -> tp.Any:
        """
        Computes classification metric (or several metrcis) for given task.
        """
        
        recall = recall_score(labels, predictions)
        precision = precision_score(labels, predictions)
        f1 = f1_score(labels, predictions)

        results = {
            'recall_score': recall,
            'precision_score': precision,
            'f1_score': f1
        }
        return results

    def save_model(self, dst_path: str) -> None:
        """
        Saves model to dst_path. Be careful to check if a model is on DataParallel state.
        If it is, one needs to process it accordingly.
        """
        if isinstance(self.model, torch.nn.DataParallel):
            torch.save(model.module.state_dict(), dst_path)
        else:
            torch.save(self.model, dst_path)

    def run_experiment(self):
        """
        Main entrypoint.
        Runs the flow from loading data to computing metrics.
        """
        wandb.login(key = 'ed5e812f0a2ec095a0e7b29e696ac3f9655e62ed')
        
          # Соберем общий конфиг для wandb
        wandb_config = {
            "epochs": 5, 
            "lr": 5e-2, 
            "weight_decay": 5e-4, 
            "learning_rate_decay_step": 2, 
            "learning_rate_decay_factor": 0.9,
            "sample_size": self.sample_size,
            "batch_size": 16
            ,}
        
        # Init для wandb
        wandb.init(project="hw-07-transformers", 
                   notes="Dataset Name " + self.dataset_name, 
                   tags=["baseline", "transformer", "distilbert-base"], 
                   config=wandb_config,
                  )
        
        train_df = self.load_data(filename = self.dataset_name, split = 'train', sample_size = self.sample_size)
   
        train_dataset = CustomDataset(
            texts=train_df["text"].tolist(),
            labels=train_df["label"].tolist(),
            tokenizer=self.tokenizer,
            max_length=512,
            
        )
      
        # Для оптимайзера - срез из конфига по wandb        
        optimizer_param_list = ['lr', 'weight_decay']
        
        optimizer_params = {k: wandb_config[k] for k in optimizer_param_list}
        self.configure_optimizer(optimizer = AdamW(self.model.parameters()), params = optimizer_params)
        
        # Выбираем экспоненциальный шедулер
        self.configure_scheduler(scheduler = ExponentialLR(optimizer=self.optimizer, gamma=wandb.config["learning_rate_decay_factor"], last_epoch=-1))
        
        self.apply_data_parallel()
        
        self.train(train_dataset)
        
        test_df = self.load_data(filename = self.dataset_name, split = 'test', sample_size = self.sample_size)
        
        test_dataset = CustomDataset(
            texts=test_df["text"].tolist(),
            labels=test_df["label"].tolist(),
            tokenizer=self.tokenizer,
            max_length=512,
        )
        
        
        valid_labels, valid_preds = self.validate(test_dataset)
        
        valid_scores = self.compute_metrics_report(valid_labels, valid_preds)
        
        self.save_model(dst_path = './distilbert-base-uncased_baseline.pt')
        
        pprint(valid_scores)
        

if __name__ == "__main__":
    """run experiment"""
    model_trainer = ModelTrainer(model_name = "distilbert-base-uncased",
                                 dataset_name = "imdb",
                                 sample_size = 5000,
                                )
    model_trainer.run_experiment()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


learning_rate,██▄▄▁
loss,█▁▁▁▁
learning_rate,0.03645
loss,0.69615


 20%|██        | 1/5 [02:23<09:32, 143.21s/it]

Epoch 1/5 - Average Loss: 2.6151 - Learning Rate 0.045000000000000005


 40%|████      | 2/5 [04:46<07:09, 143.29s/it]

...

 80%|████████  | 4/5 [09:33<02:23, 143.27s/it]

Epoch 4/5 - Average Loss: 0.6962 - Learning Rate 0.04050000000000001


100%|██████████| 5/5 [11:56<00:00, 143.27s/it]

Epoch 5/5 - Average Loss: 0.6961 - Learning Rate 0.03645000000000001
Training complete!


{'f1_score': 0.663994655978624, 'precision_score': 0.497, 'recall_score': 1.0}


In [11]:
ds = load_dataset("imdb", split = "train")
df = ds.to_pandas()

In [14]:
df["text"]

0        I rented I AM CURIOUS-YELLOW from my video sto...
1        "I Am Curious: Yellow" is a risible and preten...
2        If only to avoid making this type of film in t...
3        This film was probably inspired by Godard's Ma...
4        Oh, brother...after hearing about this ridicul...
                               ...                        
24995    A hit at the time but now better categorised a...
24996    I love this movie like no other. Another time ...
24997    This film and it's sequel Barry Mckenzie holds...
24998    'The Adventures Of Barry McKenzie' started lif...
24999    The story centers around Barry McKenzie who mu...
Name: text, Length: 25000, dtype: object